### Universidad Nacional de Lujan - Bases de Datos Masivas (11088) - Cavasin Nicolas #143501
# TP05-01 - Árboles de decisión

### Ejercicio 2:
Trabaje con el dataset de Scikit Learn “wine”:

- Utilice el metadata que provee la librería, ¿Cuál es el tema que aborda el dataset?

- Genere el árbol de decisión que permita clasificar los diferentes tipos de vino utilizando un muestreo con proporciones de 80% para entrenamiento y 20% para testeo.

- Explore la solución dada y las posibles configuraciones para obtener un nuevo árbol que clasifique “mejor”. Documente las conclusiones.

In [1]:
# Obtengo el dataset
!rm wine.data
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data

--2020-11-06 17:23:08--  https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data
Loaded CA certificate &#39;/etc/ssl/certs/ca-certificates.crt&#39;
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10782 (11K) [application/x-httpd-php]
Saving to: &#39;wine.data&#39;

wine.data           100%[===================&gt;]  10.53K  --.-KB/s    in 0.005s  

2020-11-06 17:23:11 (1.96 MB/s) - &#39;wine.data&#39; saved [10782/10782]



In [2]:
# Importo el dataset desde scikit
from sklearn.datasets import load_wine

# Lo cargo
vinos = load_wine()

# Muestro su descripcion
print(vinos.DESCR)

.. _wine_dataset:

Wine recognition dataset
------------------------

**Data Set Characteristics:**

    :Number of Instances: 178 (50 in each of three classes)
    :Number of Attributes: 13 numeric, predictive attributes and the class
    :Attribute Information:
 		- Alcohol
 		- Malic acid
 		- Ash
		- Alcalinity of ash  
 		- Magnesium
		- Total phenols
 		- Flavanoids
 		- Nonflavanoid phenols
 		- Proanthocyanins
		- Color intensity
 		- Hue
 		- OD280/OD315 of diluted wines
 		- Proline

    - class:
            - class_0
            - class_1
            - class_2
		
    :Summary Statistics:
    
    ============================= ==== ===== ======= =====
                                   Min   Max   Mean     SD
    ============================= ==== ===== ======= =====
    Alcohol:                      11.0  14.8    13.0   0.8
    Malic Acid:                   0.74  5.80    2.34  1.12
    Ash:                          1.36  3.23    2.36  0.27
    Alcalinity of Ash:            1

Se puede observar que:
- El dataset fué creado en julio de 1988 por Michael Marshall.
- Representa los resultados de un análisis químico de fermentación de vinos en la misma región de Italia realizada por tres bodegas diferentes.
- Puede ser accedido públicamente desde [aquí](https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data).
- Contiene 178 instancias de diferentes tipos de vinos separados en tres clases: *class_0*, *class_1*, *class_2*.
- Todos sus atributos son numéricos (a excepción de la descripción de cada columna) y no posee ningún valor faltante.

El valor agregado de la versión de scikit es que viene con un *poco* de pre-procesamiento. El target y los features ya se encuentran separados y los números son representados utilizado ``numpy`` en vez de *integer*s o *float*s.

In [3]:
# Importo el divisor de training y tests
from sklearn.model_selection import train_test_split

# Importo el arbol
from sklearn import tree

# Importo libreria para graficar
import graphviz

# Importo las metricas para testear el modelo
from sklearn import metrics

# Divido el dataset 80-20 como pide el enunciado
features_train, features_test, target_train, target_test = train_test_split(vinos.data, vinos.target, random_state=0, test_size=0.2)

# Instancio el arbol por entropia
t = tree.DecisionTreeClassifier(criterion='entropy')

# Lo entreno
t.fit(features_train, target_train)

# Almaceno el modelo en formato DOT
vinos_dot = tree.export_graphviz(t
                    , out_file=None
                    , feature_names=vinos.feature_names
                    , class_names=vinos.target_names
                    , filled=True, rounded=True
                    , special_characters=True)

# Tambien lo guardo como PNG
graph = graphviz.Source(vinos_dot)
graph.format = 'png'
graph.render('vinos')

# Obtengo la prediccion con el set de prueba definido anteriormente
prediccion = t.predict(features_test)

# Muestro prediccion vs original
print(f'Predicción: \t{prediccion}')
print(f'Target: \t{target_test}')
print()

# Veo qué tan acertado estuvo
print(f'Precisión del modelo: {metrics.accuracy_score(target_test, prediccion)}')
print()

# Muestro un reporte de clasificación con diferentes métricas sobre cada feature 
print(f'Reporte de clasificación: \n{metrics.classification_report(target_test, prediccion)}')


Predicción: 	[0 2 1 0 1 1 0 2 1 1 2 2 0 1 2 1 0 0 2 0 1 0 1 1 1 1 1 1 1 2 0 0 1 0 0 0]
Target: 	[0 2 1 0 1 1 0 2 1 1 2 2 0 1 2 1 0 0 1 0 1 0 0 1 1 1 1 1 1 2 0 0 1 0 0 0]

Precisión del modelo: 0.9444444444444444

Reporte de clasificación: 
              precision    recall  f1-score   support

           0       1.00      0.93      0.96        14
           1       0.94      0.94      0.94        16
           2       0.86      1.00      0.92         6

    accuracy                           0.94        36
   macro avg       0.93      0.96      0.94        36
weighted avg       0.95      0.94      0.94        36



![Vinos](vinos.png)

***Observaciones:***
- La primer solución obtenida posee una precisión del 94.4%.
- Falló solo dos veces. 
- El modelo posee una profundidad de 5 niveles.
- La hoja con mayor cantidad de sampleos posee 44, un valor algo eleveado teniendo en cuenta que hay 178 registros.
- La hoja con menor cantidad de sampleos posee 1 y se encuentra en el último nivel.


A continuación, se realizarán las siguientes pruebas para clasificar "mejor":
- Limitar el mínimo de valores de samples en hojas a 20.
- Limitar la cantidad de niveles a 3.
- Árbol que clasifique aplicando Gini y no Entropía.


In [4]:
# Instancio nuevamente el arbol por entropia pero cambio el sampleo en hojas
t2 = tree.DecisionTreeClassifier(criterion='entropy', min_samples_leaf=20)

# Lo entreno
t2.fit(features_train, target_train)

# Almaceno el modelo en formato DOT
vinos_dot = tree.export_graphviz(t2
                    , out_file=None
                    , feature_names=vinos.feature_names
                    , class_names=vinos.target_names
                    , filled=True, rounded=True
                    , special_characters=True)

# Tambien lo guardo como PNG
graph = graphviz.Source(vinos_dot)
graph.format = 'png'
graph.render('vinos_20')

# Obtengo la prediccion con el set de prueba definido anteriormente
prediccion_v2 = t2.predict(features_test)

# Muestro prediccion vs original
print(f'Predicción: \t{prediccion_v2}')
print(f'Target: \t{target_test}')
print()

# Veo qué tan acertado estuvo
print(f'Precisión del modelo: {metrics.accuracy_score(target_test, prediccion_v2)}')
print()

# Muestro un reporte de clasificación con diferentes métricas sobre cada feature 
print(f'Reporte de clasificación: \n{metrics.classification_report(target_test, prediccion_v2)}')

Predicción: 	[0 2 1 0 1 1 0 2 1 1 2 1 0 1 2 1 0 0 2 0 1 0 1 1 1 1 1 1 1 2 0 0 1 0 0 0]
Target: 	[0 2 1 0 1 1 0 2 1 1 2 2 0 1 2 1 0 0 1 0 1 0 0 1 1 1 1 1 1 2 0 0 1 0 0 0]

Precisión del modelo: 0.9166666666666666

Reporte de clasificación: 
              precision    recall  f1-score   support

           0       1.00      0.93      0.96        14
           1       0.88      0.94      0.91        16
           2       0.83      0.83      0.83         6

    accuracy                           0.92        36
   macro avg       0.91      0.90      0.90        36
weighted avg       0.92      0.92      0.92        36



![Vinos](vinos_20.png)

In [5]:
# Instancio nuevamente el arbol por entropia pero cambio la profundidad (de 0 a 2)
t3 = tree.DecisionTreeClassifier(criterion='entropy', max_depth=2)

# Lo entreno
t3.fit(features_train, target_train)

# Almaceno el modelo en formato DOT
vinos_dot = tree.export_graphviz(t3
                    , out_file=None
                    , feature_names=vinos.feature_names
                    , class_names=vinos.target_names
                    , filled=True, rounded=True
                    , special_characters=True)

# Tambien lo guardo como PNG
graph = graphviz.Source(vinos_dot)
graph.format = 'png'
graph.render('vinos_3_niveles')

# Obtengo la prediccion con el set de prueba definido anteriormente
prediccion_v3 = t3.predict(features_test)

# Muestro prediccion vs original
print(f'Predicción: \t{prediccion_v3}')
print(f'Target: \t{target_test}')
print()

# Veo qué tan acertado estuvo
print(f'Precisión del modelo: {metrics.accuracy_score(target_test, prediccion_v3)}')
print()

# Muestro un reporte de clasificación con diferentes métricas sobre cada feature 
print(f'Reporte de clasificación: \n{metrics.classification_report(target_test, prediccion_v3)}')

Predicción: 	[0 2 1 0 1 1 0 2 1 1 2 2 0 1 2 1 0 0 2 0 1 0 1 1 1 1 1 1 1 2 0 0 1 0 0 0]
Target: 	[0 2 1 0 1 1 0 2 1 1 2 2 0 1 2 1 0 0 1 0 1 0 0 1 1 1 1 1 1 2 0 0 1 0 0 0]

Precisión del modelo: 0.9444444444444444

Reporte de clasificación: 
              precision    recall  f1-score   support

           0       1.00      0.93      0.96        14
           1       0.94      0.94      0.94        16
           2       0.86      1.00      0.92         6

    accuracy                           0.94        36
   macro avg       0.93      0.96      0.94        36
weighted avg       0.95      0.94      0.94        36



![Vinos](vinos_3_niveles.png)

In [6]:
# Instancio nuevamente el arbol pero para que clasifique por Gini
t4 = tree.DecisionTreeClassifier(criterion='gini')

# Lo entreno
t4.fit(features_train, target_train)

# Almaceno el modelo en formato DOT
vinos_dot = tree.export_graphviz(t4
                    , out_file=None
                    , feature_names=vinos.feature_names
                    , class_names=vinos.target_names
                    , filled=True, rounded=True
                    , special_characters=True)

# Tambien lo guardo como PNG
graph = graphviz.Source(vinos_dot)
graph.format = 'png'
graph.render('vinos_gini')

# Obtengo la prediccion con el set de prueba definido anteriormente
prediccion_v4 = t4.predict(features_test)

# Muestro prediccion vs original
print(f'Predicción: \t{prediccion_v4}')
print(f'Target: \t{target_test}')
print()

# Veo qué tan acertado estuvo
print(f'Precisión del modelo: {metrics.accuracy_score(target_test, prediccion_v4)}')
print()

# Muestro un reporte de clasificación con diferentes métricas sobre cada feature 
print(f'Reporte de clasificación: \n{metrics.classification_report(target_test, prediccion_v4)}')

Predicción: 	[0 2 1 0 1 1 0 2 1 1 2 2 0 1 2 1 0 0 1 0 0 0 0 1 1 1 1 1 1 2 0 0 1 0 0 0]
Target: 	[0 2 1 0 1 1 0 2 1 1 2 2 0 1 2 1 0 0 1 0 1 0 0 1 1 1 1 1 1 2 0 0 1 0 0 0]

Precisión del modelo: 0.9722222222222222

Reporte de clasificación: 
              precision    recall  f1-score   support

           0       0.93      1.00      0.97        14
           1       1.00      0.94      0.97        16
           2       1.00      1.00      1.00         6

    accuracy                           0.97        36
   macro avg       0.98      0.98      0.98        36
weighted avg       0.97      0.97      0.97        36



![Arbol](vinos_gini.png)

***Observaciones:***
- La versión 2, con un límite mínimo de 20 samples por hoja, tiene menos precisión en la predicción con respecto a la versión original: 91.6% vs 94.4% respectivamente.

- La versión 3, con un límite de profundidad de 3 niveles, mantuvo la misma precisión que el modelo original pero se "ahorró" 2 niveles. Probablemente si se destinara un set de test mayor al 20% que solicita la consigna, la precisión decaiga.

- La versión 4, que clasifica utilizando el Nivel de Pureza de un dato (Gini) y no la Ganancia de Información (Entropía), fué el que mayor precisión brindó: 97.2%.

## Conclusiones:

- Según los resultados de *este* dataset, la clasificación aplicando el método Gini fué la más precisa de las 4 exploradas, es decir, la "mejor".

- Eso no significa que sea una regla general. Sino que, dado este dataset de vinos, dicho método es el que con mayor precisión predice.